In [1]:
import numpy as np
import plotting as pm
import os

from vehicle_reconstruction import vehicle, car_models, vehicle_reconstructor

CAR_MODEL_DIR = "F:\\ApolloScape\\3d_car_instance_sample\\3d_car_instance_sample\\car_models_json"

: 

In [ ]:
vehicles: list[vehicle] = vehicle.load_car_models(car_model_dir=CAR_MODEL_DIR,
                                models=car_models.models)

for id, vehi in enumerate(vehicles):
    print(f"\t{id}:\t{vehi.name:25}", end='\t')
    if id % 4 == 0:
        print()

In [ ]:
if not os.path.exists(".\\vehicles_original"):
    os.mkdir(".\\vehicles_original")

for vehi in vehicles:
    pm.create_figure_for_inspection(
        [vehi.vertices[:, [0,2,1]]], [vehi.faces], f".\\vehicles_original\\{vehi.name}.html")

In [ ]:
idx = 60
print(vehicles[idx])
vehicles[idx].to_trimesh().show()

In [ ]:
global_res = 0.1
standard_bbox = [2.5, 1.9, 5.6, -1.25, 1.25, 0.0, 1.90, -2.8, 2.8]
grid_res = [int(np.ceil(standard_bbox[0]/global_res)),
            int(np.ceil(standard_bbox[1]/global_res)),
            int(np.ceil(standard_bbox[2]/global_res))]
print(grid_res)
print(vehicles[idx])
vehicles[idx].to_voxel(sampling_space=standard_bbox, 
                       grid_res=grid_res).to_trimesh().show()

In [ ]:
vehi_reconstructor = vehicle_reconstructor(vehicles=vehicles, 
                      sampling_space=standard_bbox, 
                      grid_res=grid_res)

In [ ]:
use_preprocessed_voxel = True

if use_preprocessed_voxel:
    vehi_reconstructor.load_voxel(".\\preprocessed_voxel.bin")
else:
    vehi_reconstructor.voxelize_vehicles()
    vehi_reconstructor.save_voxel(".\\preprocessed_voxel.bin")

In [ ]:
vehi_reconstructor.fit_model(k=5)

In [ ]:
reconstructed_vehicles = vehi_reconstructor.reconsturct()

In [ ]:
reconstructed_vehicles[idx].to_trimesh().show()

In [ ]:
if not os.path.exists(".\\vehicles_reconstructed"):
    os.mkdir(".\\vehicles_reconstructed")

for idx, vehi in enumerate(reconstructed_vehicles):
    vehi = vehi.to_trimesh()
    pm.create_figure_for_inspection(
        [vehi.vertices[:, [0, 2, 1]]], [vehi.faces], f".\\vehicles_reconstructed\\{vehicles[idx].name}.html")

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider

global vehi
vehi = None

def slidar_create(min=-10.0, max=10.0, desc="desc:"):
    return widgets.FloatSlider(
        value=0.0,
        min=min,
        max=max,
        step=0.1,
        description=desc,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='.1f',
    )


def generate_model(a, b, c, d, e):
    latent = torch.Tensor([[a, b, c, d, e]])
    global vehi
    vehi = vehi_reconstructor.decode(latent)[0].to_trimesh()
    # pm.create_figure_for_inspection(
    #     [vehi.vertices[:, [0, 2, 1]]], [vehi.faces], f"temp.html")

interact(generate_model, a=slidar_create(-40.0, 40.0, 'v'),
         b=slidar_create(-20.0, 20.0, 'w'),
         c=slidar_create(-20.0, 20.0, 'x'),
         d=slidar_create(-20.0, 20.0, 'y'),
         e=slidar_create(-20.0, 20.0, 'z'))

In [ ]:
vehi.show()

: 